In [1]:
#Install pip packages in the curret Jupyter kernal
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade pyodbc
!{sys.executable} -m pip install polars

  Using cached pip-23.2-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ---------------------------------------- 0.0/69.7 kB ? eta -:--:--
     ---------------------------------------- 69.7/69.7 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: pyodbc
    Found existing installation: pyodbc 4.0.34
    Uninstalling pyodbc-4.0.34:
      Successfully uninstalled pyodbc-4.0.34
  Obtaining dependency information for polars from https://files.pythonhosted.org/packages/c5/18/4f0fbacc2f04eb1ef64cb7fe452d849d32c17bb10543391cfe7ac0cd4f9b/polars-0.18.7-cp38-abi3-win_amd64.whl.metadata
  Using cached polars-0.18.7-cp38-abi3-win_amd64.whl.metadata (14 kB)
Using cached polars-0.18.7-cp38-abi3-win_amd64.whl (19.8 MB)


Notes on behavior:

Byes do not count towards OMW and OGW one way or othe other.

If a bye is assigned, it's first assigned among subjects with the fewest byes (usually zero) and second among subjects with the fewest points (should be updated to lowest overall ranking).

I've calculated omw, gw, and ogw wrong. They're fractions of points, not fractions of wins. See https://media.wizards.com/2023/wpn/marketing_materials/wpn/mtg_mtr_2023may29_en.pdf



In [9]:
import random
import polars as pl
import networkx as nx

In [72]:
def pop_random(lst):
    idx = random.randrange(0, len(lst))
    return lst.pop(idx)

wld_list = ['W'] * 45 + ['L'] * 45 + ['D'] * 10

def match_dict(n):
    lst = ["Player " + str(i) for i in range(n)]
    pairs = []
    pair_dict = {}
    point_dict = {}
    while lst:
        rand1 = pop_random(lst)
        rand2 = pop_random(lst)
        pair = [rand1, rand2]
        pairs.append(pair)
    for pair in pairs:
        pair_dict[pair[0]] = pair[1]
        pair_dict[pair[1]] = pair[0]
        wld = random.choice(wld_list)
        if wld == "W":
            point_dict[pair[0]] = 3
            point_dict[pair[1]] = 0
        elif wld == "L":
            point_dict[pair[0]] = 0
            point_dict[pair[1]] = 3
        elif wld == "D":
            point_dict[pair[0]] = 1
            point_dict[pair[1]] = 1
        
    return pair_dict, point_dict

In [20]:
graph_players = {"A": "B", "B": "A", "C": "D", "D": "C", "E": []}
graph_points = {"A": 3, "B": 0, "C": 1, "D": 1, "E": 3}

In [90]:
graph_players, graph_points = match_dict(200)

In [94]:
bracketGraph = nx.Graph()
bracketGraph.add_nodes_from(graph_players)
for player in bracketGraph.nodes():
    for opponent in bracketGraph.nodes():
        if opponent not in graph_players[player] and player != opponent:
            #wgt = random.randint(1, 9) #randomizes pairings. Not necessarily desirable.
            wgt = 1 + (graph_points[player] + graph_points[opponent])**2
            bracketGraph.add_edge(player, opponent, weight=wgt)
pairings_bg = dict(nx.max_weight_matching(bracketGraph))
len(pairings_bg)

100

In [95]:
for key, value in pairings_bg.items():
    if graph_points[key] != graph_points[value]:
        print(graph_points[key], graph_points[value])
        print(key, value)

1 0
Player 146 Player 184
3 1
Player 43 Player 109


In [93]:
graph_points

{'Player 191': 0,
 'Player 43': 3,
 'Player 134': 0,
 'Player 174': 3,
 'Player 145': 3,
 'Player 178': 0,
 'Player 146': 1,
 'Player 0': 1,
 'Player 12': 0,
 'Player 59': 3,
 'Player 111': 0,
 'Player 10': 3,
 'Player 176': 0,
 'Player 39': 3,
 'Player 129': 0,
 'Player 128': 3,
 'Player 80': 3,
 'Player 36': 0,
 'Player 179': 3,
 'Player 33': 0,
 'Player 52': 0,
 'Player 55': 3,
 'Player 194': 3,
 'Player 50': 0,
 'Player 136': 3,
 'Player 8': 0,
 'Player 173': 3,
 'Player 164': 0,
 'Player 163': 3,
 'Player 103': 0,
 'Player 113': 0,
 'Player 138': 3,
 'Player 13': 3,
 'Player 139': 0,
 'Player 161': 1,
 'Player 86': 1,
 'Player 15': 1,
 'Player 127': 1,
 'Player 6': 0,
 'Player 135': 3,
 'Player 64': 0,
 'Player 169': 3,
 'Player 14': 3,
 'Player 155': 0,
 'Player 49': 3,
 'Player 65': 0,
 'Player 171': 0,
 'Player 60': 3,
 'Player 83': 3,
 'Player 196': 0,
 'Player 124': 0,
 'Player 87': 3,
 'Player 95': 3,
 'Player 125': 0,
 'Player 53': 0,
 'Player 24': 3,
 'Player 112': 0,
 'Pl

In [3]:
players = ["Esben",
          "Jeppe",
          "Halfdan",
          "Asger",
          "Bakke"]
if (len(players) > len(set(players))):
    raise ValueError('There are non-unique players.')
elif ("Bye" in players):
    raise ValueError('You have listed "Bye" as a player. Please don\'t.')

players_df = pl.DataFrame({"player": players, "points": 0, "match_wins": 0, "game_wins": 0, "total_matches": 0, "total_games": 0, "opponents": "", "omw": 0., "gw": 0., "ogw": 0., "dropped": False})
players_df

player,points,match_wins,game_wins,total_matches,total_games,opponents,omw,gw,ogw,dropped
str,i64,i64,i64,i64,i64,str,f64,f64,f64,bool
"""Esben""",0,0,0,0,0,"""""",0.0,0.0,0.0,false
"""Jeppe""",0,0,0,0,0,"""""",0.0,0.0,0.0,false
"""Halfdan""",0,0,0,0,0,"""""",0.0,0.0,0.0,false
"""Asger""",0,0,0,0,0,"""""",0.0,0.0,0.0,false
"""Bakke""",0,0,0,0,0,"""""",0.0,0.0,0.0,false


In [4]:
def pairings(pair_players):
    
    pair_players = pair_players.filter(pl.col("dropped") == False)
    
    has_bye = len(pair_players) % 2 == 1
    
    if (has_bye):
        bye_players = pair_players.with_columns(
            bye_count = pl.col("opponents").str.count_match("Bye")
        ).filter(
            pl.col("bye_count") == pl.col("bye_count").min()
        ).filter(
            pl.col("points") == pl.col("points").min()
        )
        
        bye_pair = bye_players.sample(1).select("player").vstack(
                pl.DataFrame({"player": "Bye"})
            ).with_columns(
                match = 0,
                play_draw = pl.Series(["play", "draw"])
            )
        
        pair_players0 = pair_players = pair_players.join(bye_pair, on = "player", how = "anti").with_columns(forbidden = pl.lit(""))
                
    pairs_n = len(pair_players)/2
    
    pairs = pair_players.select("player").with_columns(
        match = 0,
        play_draw = pl.lit("play")
    ).clear()
    
    n=0
    while (len(pair_players) > 0):
        n = n+1
        
        match_player_1 = pair_players.slice(0, 1)

        unmatch_list = match_player_1.with_columns(
            oppo_struct = pl.col("opponents").str.split("//").list.to_struct()
        ).select("oppo_struct").unnest("oppo_struct").melt(
            id_vars = None, value_vars = None, value_name = "player"
        ).groupby("player").agg(pl.count()).filter(
            pl.col("count") == pl.col("count").max()
        ).drop("count")

        match_player_2 = pair_players.select("player").join(
            unmatch_list, on = "player", how = "anti"
        ).join(
            match_player_1, on = "player", how = "anti"
        ).slice(0, 1)
       
        pair = match_player_1.select("player").vstack(match_player_2)
        
        if (len(pair) != 2):
            pair_players = pair_players.clear()
            pairs = pairs.clear()
            continue
        
        pair_players = pair_players.join(pair, on = "player", how = "anti")
        
        pair = pair.with_columns(
            match = n,
            play_draw = pl.Series(random.sample(["play", "draw"], 2))
        )
        
        pairs = pairs.vstack(pair)
            
    if (has_bye):
        pairs = bye_pair.vstack(pairs)
            
    pairs = pairs.pivot(
        values="player", index="match", columns="play_draw", aggregate_function="first"        
    ).select(["match", "play", "draw"])
    
    return pairs

pairings(players_df)

match,play,draw
i32,str,str
0,"""Halfdan""","""Bye"""
1,"""Jeppe""","""Esben"""
2,"""Bakke""","""Asger"""


In [5]:
pairings_1 = pairings(players_df)
pairings_1

match,play,draw
i32,str,str
0,"""Asger""","""Bye"""
1,"""Jeppe""","""Esben"""
2,"""Bakke""","""Halfdan"""


In [6]:
def results(players_df, pairings_df, result_list = None, drop_list = None):
    
    #TODO: Accept manual results
    if (result_list is None):
        result_list = input("How did it go? ")
        
    if (len(pairings_df.filter(pl.col("play") == "Bye")) > 0):
        result_list = result_list + ["0-2"]
    elif (len(pairings_df.filter(pl.col("draw") == "Bye")) > 0):
        result_list = result_list + ["2-0"]
    
    result_df = pl.DataFrame({"result": result_list}).with_columns(
        wins = pl.col("result").str.split_exact("-", 2)
    ).unnest("wins").rename(
        {"field_0": "play_wins", "field_1": "draw_wins", "field_2": "games_drawn"}
    ).with_columns(
        play_wins = pl.col("play_wins").cast(pl.Int32),
        draw_wins = pl.col("draw_wins").cast(pl.Int32),
        games_drawn = pl.col("games_drawn").cast(pl.Int32),
        play_points = pl.when(pl.col("play_wins") > pl.col("draw_wins")).then(pl.lit(3))\
            .when(pl.col("play_wins") == pl.col("draw_wins")).then(pl.lit(1))\
            .otherwise(pl.lit(0)),
        draw_points = pl.when(pl.col("draw_wins") > pl.col("play_wins")).then(pl.lit(3))\
            .when(pl.col("draw_wins") == pl.col("play_wins")).then(pl.lit(1))\
            .otherwise(pl.lit(0))
    ).fill_null(0).with_columns(
        total_games = pl.col("play_wins") + pl.col("draw_wins") + pl.col("games_drawn")
    )
    
    result_df = pairings_df.hstack(result_df)
    
    play_df = result_df.select(pl.col("^play_.*$"), player = "play", new_opponent = "draw", new_games = "total_games")
    draw_df = result_df.select(pl.col("^draw_.*$"), player = "draw", new_opponent = "play", new_games = "total_games")

    play_df.columns = list(map(lambda x: x.replace("play_", "new_"), play_df.columns))
    draw_df.columns = list(map(lambda x: x.replace("draw_", "new_"), draw_df.columns))
    
    update_df = play_df.vstack(draw_df).filter(pl.col("player") != "Bye").with_columns(
        new_victory = pl.when(pl.col("new_points") == 3).then(pl.lit(1)).otherwise(pl.lit(0))
    )
    
    players_df = players_df.drop(["gw", "omw", "ogw"]).join(update_df, on = "player", how = "left").with_columns(
        points = pl.col("points") + pl.col("new_points"),
        match_wins = pl.col("match_wins") + pl.col("new_victory"),
        game_wins = pl.col("game_wins") + pl.col("new_wins"),
        total_matches = pl.col("total_matches") + pl.lit(1),
        total_games = pl.col("total_games") + pl.col("new_games"),
        opponents = (pl.col("opponents") + pl.lit("//") + pl.col("new_opponent")).str.replace("^//", "")
    ).select(pl.col("^[^n].*$")).with_columns(
        oppo_struct = pl.col("opponents").str.split("//").list.to_struct()
    )
    
    player_opponent_df = players_df.unnest("oppo_struct").select(pl.col("^field.*$"), "player").melt(
        id_vars = "player", value_vars = None, value_name = "opponent"
    ).drop("variable").join(
        players_df.drop(["points", "opponents", "oppo_struct"]), left_on = "opponent", right_on = "player", how = "inner"
    ).groupby("player").sum().drop("opponent").with_columns(
        omw = 100*pl.col("match_wins")/pl.col("total_matches"),
        ogw = 100*pl.col("game_wins")/pl.col("total_games")
    ).select(["player", "omw", "ogw"])
    
    players_df = players_df.join(
        player_opponent_df, on = "player", how = "left"
    ).with_columns(
        gw = 100*pl.col("game_wins")/pl.col("total_games")
    ).sort(["points", "omw", "gw", "ogw"], descending = True).drop("oppo_struct").with_columns(
        dropped = pl.when(pl.col("player").is_in(drop_list)).then(pl.lit(True)).otherwise(pl.lit(False))
    )
    
    return players_df

#players_df_2 = results(players_df_1, pairings_2, ["2-1", "1-1"], ["Jeppe", "Asger"])
#players_df_2
#results(players_df, pairings_1, ["2-1", "1-1-1"])
#results(pairings_1)

In [8]:
players_df_1 = results(players_df, pairings_1, ["2-1", "1-1-1"])
pairings_2 = pairings(players_df_1)
pairings_2

match,play,draw
i32,str,str
0,"""Halfdan""","""Bye"""


In [500]:
print(pairings_1)
print(pairings_2)

shape: (3, 3)
┌───────┬───────┬─────────┐
│ match ┆ play  ┆ draw    │
│ ---   ┆ ---   ┆ ---     │
│ i32   ┆ str   ┆ str     │
╞═══════╪═══════╪═════════╡
│ 1     ┆ Esben ┆ Halfdan │
│ 2     ┆ Bakke ┆ Jeppe   │
│ 3     ┆ Bye   ┆ Asger   │
└───────┴───────┴─────────┘
shape: (3, 3)
┌───────┬───────┬─────────┐
│ match ┆ play  ┆ draw    │
│ ---   ┆ ---   ┆ ---     │
│ i32   ┆ str   ┆ str     │
╞═══════╪═══════╪═════════╡
│ 1     ┆ Asger ┆ Halfdan │
│ 2     ┆ Esben ┆ Bakke   │
│ 3     ┆ Jeppe ┆ Bye     │
└───────┴───────┴─────────┘


In [387]:
print('WARNING: this cell uses XXXX limited resources:')
answer = input('-> Are you sure you want to run it? [yes| no]: ' )
if answer == 'yes':
    print("YES")
else:
    print("NO")
print("some other job")

-> Are you sure you want to run it? [yes| no]:  a


NO
some other job
